# Выбор региона для разработки новых нефтяных месторождений

# Описание проекта

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 скважин, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Также проведём анализ возможной прибыли и рисков техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают скважины с самыми высокими оценками значений. Количество скважин зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных скважин.

# 1. Загрузка и подготовка данных

#### Описание данных

+ id — уникальный идентификатор скважины;
+ f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
+ product — объём запасов в скважине (тыс. баррелей).

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [2]:
geo0 = pd.read_csv('/datasets/geo_data_0.csv')
geo1 = pd.read_csv('/datasets/geo_data_1.csv')
geo2 = pd.read_csv('/datasets/geo_data_2.csv')

regions = [geo0, geo1, geo2]

for region in regions:
    region.info()
    print()

geo0.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


+ Пропусков нет
+ Дубликаты не обнаружены
+ По всей видимости признаки уже подготовлены к использованию, предобработка не требуется.

# 2. Обучение и проверка модели

##### Разделение данных

Разделяем данные по каждому региону на две выборки: тренировочную (75%) и валидационную(25%)

In [3]:
features_0 = geo0.drop(['id', 'product'], axis=1)
target_0 = geo0['product']
features_1 = geo1.drop(['id', 'product'], axis=1)
target_1 = geo1['product']
features_2 = geo2.drop(['id', 'product'], axis=1)
target_2 = geo2['product']


features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
                                                                        features_0, target_0, test_size=0.25, random_state=42)
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
                                                                        features_1, target_1, test_size=0.25, random_state=42)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
                                                                        features_2, target_2, test_size=0.25, random_state=42)

##### Обучение и проверка моделей

In [4]:
def prediction(features_train, target_train, features_valid, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = pd.Series(model.predict(features_valid))
    reserves_mean = predicted_valid.mean()
    rmse = mean_squared_error(target_valid, predicted_valid) ** 0.5
    return reserves_mean, rmse, predicted_valid

In [5]:
def printing(reserves_mean, rmse, region):
    print('Регион', region)
    print('Среднее значение предсказанных запасов: {:.2f}'.format(reserves_mean))
    print('RMSE = {:.2f}'.format(rmse))
    print()

In [6]:
reserves_mean_0, rmse_0, predicted_valid_0 = prediction(features_train_0, target_train_0, features_valid_0, target_valid_0)
reserves_mean_1, rmse_1, predicted_valid_1 = prediction(features_train_1, target_train_1, features_valid_1, target_valid_1)
reserves_mean_2, rmse_2, predicted_valid_2 = prediction(features_train_2, target_train_2, features_valid_2, target_valid_2)

printing(reserves_mean_0, rmse_0, 0)
printing(reserves_mean_1, rmse_1, 1)
printing(reserves_mean_2, rmse_2, 2)

Регион 0
Среднее значение предсказанных запасов: 92.40
RMSE = 37.76

Регион 1
Среднее значение предсказанных запасов: 68.71
RMSE = 0.89

Регион 2
Среднее значение предсказанных запасов: 94.77
RMSE = 40.15



Регион 1 резко отличается от двух других очень низким RMSE, что говорит о высоком качестве модели. Вероятно, это обусловлено более закономерным распределением изучаемых признаков. Предсказания по регионам 0 и 2 уже не такие надёжные.

# 3. Подготовка к расчёту прибыли

In [7]:
budget = 10 * (10 ** 9)
earnings = 450 * (10 ** 3)
n_wells = 500
n_best_wells = 200

+ Бюджет на разработку скважин 10 млрд рублей
+ Доход на единицу продукции (тыс. баррелей) 450 тысяч рублей
+ Количество скважин разбуриваемых при разведке региона 500 штук
+ Количество скважин выбираемых для подсчёта прибыли 200 штук

In [8]:
break_even = budget / earnings / n_best_wells
break_even

111.11111111111111

Для безубыточной разработки новой скважины запасы нефти в ней должны превышать 111.11 тысяч баррелей

In [9]:
i = 0
for region in regions:
    print('Регион {:.0f}: средние запасы в скважине составляют {:.2f} тыс. баррелей'.format(i, region['product'].mean()))
    i += 1

Регион 0: средние запасы в скважине составляют 92.50 тыс. баррелей
Регион 1: средние запасы в скважине составляют 68.83 тыс. баррелей
Регион 2: средние запасы в скважине составляют 95.00 тыс. баррелей


Средние запасы в скважинах по всем регионам не дотягивают до точки безубыточности. Выходит, что использовать все скважины в регионе для разработки невыгодно, нужно выбирать только лучшие.
(Также отмечу, что истинные средние значения почти не отличаются от предсказанных)

# 4. Расчёт прибыли и рисков 

In [10]:
def profit_calc(target, predictions):
    best_wells = predictions.sort_values(ascending=False)[:n_best_wells]
    indices_list = list(best_wells.index)
    selected_target = target.reset_index(drop=True).loc[indices_list]
    revenue = selected_target.sum() * earnings
    profit = revenue - budget
    return profit

In [11]:
def risk_profit(target, predictions, n):
    state = np.random.RandomState(42)
    values = []
    for i in range(n):
        subsample = predictions.sample(n=n_wells, replace=True, random_state=state)
        value = profit_calc(target, subsample)
        values.append(value)
        
    values = pd.Series(values)
    risk = values[values < 0].count() / n
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    print('95% доверительный интервал: ({:.0f}, {:.0f})'.format(lower, upper))
    print('Среднее значение прибыли: {:.0f} рублей'.format(values.mean()))
    print('Риск понести убытки: {:.1%}'.format(risk))
    print()

In [12]:
print('Регион 0')
risk_profit(target_valid_0, predicted_valid_0, 1000)

print('Регион 1')
risk_profit(target_valid_1, predicted_valid_1, 1000)

print('Регион 2')
risk_profit(target_valid_2, predicted_valid_2, 1000)

Регион 0
95% доверительный интервал: (-110467895, 897460328)
Среднее значение прибыли: 399575478 рублей
Риск понести убытки: 6.0%

Регион 1
95% доверительный интервал: (61684480, 845340178)
Среднее значение прибыли: 452048891 рублей
Риск понести убытки: 1.5%

Регион 2
95% доверительный интервал: (-144766727, 888390404)
Среднее значение прибыли: 375009903 рублей
Риск понести убытки: 8.0%



Под требуемые условия подходит лишь Регион 1, у которого риск убытков менее 2.5% и составляет всего лишь 1.5%.  
Что любопытно, при более низком среднем значении запасов в каждой скважине в Регионе 1, нежели в других регионах, средняя возможная прибыль выше. Это ещё раз подтверждает, что все метрики нужно использовать аккуратно и полностью на них не полагаться.